In [ ]:
import numpy as np
import rasterio as rio
import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon
from cartopy.feature import ShapelyFeature
import matplotlib.patches as mpatches


def percentile_stretch(img, pmin=0., pmax=100.):
    '''
    This is where you should write a docstring.
    '''
    # here, we make sure that pmin < pmax, and that they are between 0, 100
    if not 0 <= pmin < pmax <= 100:
        raise ValueError('0 <= pmin < pmax <= 100')
    # here, we make sure that the image is only 2-dimensional
    if not img.ndim == 2:
        raise ValueError('Image can only have two dimensions (row, column)')

    minval = np.percentile(img, pmin)
    maxval = np.percentile(img, pmax)

    stretched = (img - minval) / (maxval - minval)  # stretch the image to 0, 1
    stretched[img < minval] = 0  # set anything less than minval to the new minimum, 0.
    stretched[img > maxval] = 1  # set anything greater than maxval to the new maximum, 1.

    return stretched


def img_display(image, ax, bands, transform, extent, pmin=0, pmax=100):
    '''
    This is where you should write a docstring.
    '''
    dispimg = image.copy().astype(np.float32) # make a copy of the original image,
    # but be sure to cast it as a floating-point image, rather than an integer

    for b in range(image.shape[0]): # loop over each band, stretching using percentile_stretch()
        dispimg[b] = percentile_stretch(image[b], pmin=pmin, pmax=pmax)

    # next, we transpose the image to re-order the indices
    dispimg = dispimg.transpose([1, 2, 0])
    
    # finally, we display the image
    handle = ax.imshow(dispimg[:, :, bands], transform=transform, extent=extent)
    
    return handle, ax

# ------------------------------------------------------------------------
# note - rasterio's open() function works in much the same way as python's - once we open a file,
# we have to make sure to close it. One easy way to do this in a script is by using the with statement shown
# below - once we get to the end of this statement, the file is closed.
with rio.open('data_files/NI_Mosaic.tif') as dataset:
    img = dataset.read()
    xmin, ymin, xmax, ymax = dataset.bounds

    # your code goes here!
    # start by loading the outlines and point data to add to the map
    counties4326 = gpd.read_file('data_files/Counties.shp') # load the Counties shapefile
    towns4326 = gpd.read_file('data_files/Towns.shp') # load the Towns shapefile

    counties = counties4326.to_crs(epsg=32629)
    towns = towns4326.to_crs(epsg=32629)
    
    print(counties.crs)
    print(towns.crs)

    # next, create the figure and axis objects to add the map to
    ni_utm = ccrs.UTM(29) # note that this matches with the CRS of our image
    fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=ni_utm))

    # now, add the satellite image to the map
    h, ax = img_display(img, ax, [2, 1, 0], ni_utm, [xmin, xmax, ymin, ymax], pmin=0.1, pmax=99.9)
    
    # next, add the county outlines to the map
    county_outlines = ShapelyFeature(counties['geometry'], ni_utm, edgecolor='r', facecolor='none')
    ax.add_feature(county_outlines)

    print(dataset.crs)

    # then, add the town and city points to the map, but separately
    acity = towns[towns['STATUS'] == 'City']
    atown = towns[towns['STATUS'] == 'Town']

    city_handle = ax.plot(acity.geometry.x, acity.geometry.y, 's', color='red', ms=6)
    town_handle = ax.plot(atown.geometry.x, atown.geometry.y, 'o', color='blue', ms=6)
    
    # finally, try to add a transparent overlay to the map
    # note: one way you could do this is to combine the individual county shapes into a single shape, then
    # use a geometric operation, such as a symmetric difference, to create a hole in a rectangle.
    # then, you can add the output of the symmetric difference operation to the map as a semi-transparent feature.

    # Combine individual county shapes into a single shape
    combined_counties = unary_union(counties['geometry'])

    # Create a rectangular polygon representing the extent of the image
    image_extent_polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)])

    # Create a hole in the rectangular polygon using symmetric difference with combined counties
    transparent_overlay_shape = image_extent_polygon.symmetric_difference(combined_counties)

    # Convert the resulting shape into a ShapelyFeature
    transparent_overlay = ShapelyFeature([transparent_overlay_shape], ni_utm, edgecolor=None, facecolor='grey', alpha=0.7)

    # Add the transparent overlay to the map
    ax.add_feature(transparent_overlay)

    # last but not least, add gridlines to the map
    gridlines = ax.gridlines(draw_labels = False,  # draw labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5],  # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5])  # add latitude lines at 0.5 deg intervals
    gridlines.left_labels = True  # turn off the left-side labels
    gridlines.bottom_labels = True  # turn on the bottom labels
    gridlines.top_labels = False
    gridlines.right_labels = False
    
    # show the figure
    fig

    # and of course, save the map!
    fig.savefig('P4map.png', bbox_inches='tight', dpi=300)
   